Introduction to Data Science and Intel DAAL
===========================================


Raphael Mendes de Oliveira Cóbe

rmcobe@ncc.unesp.br

# NCC/Unesp

* GridUnesp
    * 256 WorkerNodes
    
* Sprace
    * Tier-2 Cern
    * 1PB Raw Data

# Data Science

* Recently open field that put together lots of concepts of other fields, such as:
  * Data Mining;
  * Data Cleaning;

# Intel Data Analytics Acceleration Library
* Delivers high application performance across spectrum of Intel®-architecture devices
* Speeds time-to-value through data source and environment integration 
* Reduces application development time via wide selection of pre-optimized advanced analytics algorithms

# Data Management

* Data Table
<center><img src="datatable.png" width=400 style="transform:rotate(90deg);"/></center>

# Data Management and Data Structures

* **Heterogeneous Tables** are used when there are multiple data types in a data set (e.g.double, string, etc.) 
    * Structures supported: **Structures of Arrays (SoA)** and **Arrays of Structures (AoS)**. 
* **Homogeneous Tables** are used when the data set has only one type of data. 
    * Two types of homogeneous tables: **dense** and **sparse**. 
* **Matrices** are used when the application requires matrix algebra type workloads
    * Three kinds of matrices supported: **dense matrix**, **packed symmetric matrix**, and **packed triangular matri-ces**. 




# AOS vs SOA

<center><img src="aos.png"/></center>
<center><img src="soa.png"/></center>

# Matrices
<center><img src="condensed-matrix.png" style="height:300px;"/></center>
<center><img src="symmetric.png" style="height:400px;"/></center>

# What is included?

* Algorithms for:
    * **Pre-processing**, **Transformation**, **Analysis**, **Modeling**.

<center><img src="intel-daal-all-data-analysis-stages.png"/></center>